<img src="images/strathsdr_banner.png" align="left">

# Hardware Accelerated Spectrum Analysis on RFSoC
----

<div class="alert alert-box alert-info">
Please use Jupyter Labs http://board_ip_address/lab for this notebook.
</div>

This notebook presents a flexible hardware accelerated Spectrum Analyzer Module for the Zynq UltraScale+ RFSoC. The Spectrum Analyzer Module was developed by the [University of Strathclyde](https://github.com/strath-sdr).

## Table of Contents
* [Introduction](#introduction)
    * [Hardware Setup](#hardware-setup)
    * [Software Setup](#software-setup)
* [Simple Tone Generation](#simple-tone-generation)
* [The Spectrum Analyzer](#the-spectrum-analyzer)
    * [A Simple Example](#a-simple-example)
* [Conclusion](#conclusion)
    
## References
* [Xilinx, Inc, "USP RF Data Converter: LogiCORE IP Product Guide", PG269, v2.4, November 2020](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_4/pg269-rf-data-converter.pdf)

## Revision History
* **v1.0** | 12/02/2021 | Spectrum analyzer notebook
* **v1.1** | 15/04/2021 | Update spectral resolution and minimum bandwidth with new value
* **v1.2** | 31/03/2022 | Update description of spectrum analyser

----

## Introduction <a class="anchor" id="introduction"></a>
The Zynq RFSoC contains high frequency samplers known as RF Data Converters (RF DCs). The RF DCs are tightly coupled with the Programmable Logic (PL), creating a high-throughput, low-latency path between the FPGA and analogue world. The Spectrum Analyzer Module employs the RF Analogue-to-Digital Converters (RF ADCs) to receive RF time domain signals. The received data is manipulated using spectral pre-processing techniques in the PL, to prepare it for frequency domain analysis and visualisation in the Processing System (PS).

A significant portion of the design has been implemented in the RFSoC's PL to prevent the PS from applying highly computational arithemtic. There is a Spectrum Analyzer Module for each available RF ADC channel in the design. The Spectrum Analyzers are also interfaced to their very own flexible decimator, allowing different sample rates to be configured for each channel.

### Hardware Setup <a class="anchor" id="hardware-setup"></a>
Your RFSoC2x2 development board can host two Spectrum Analyzer Modules. There are four SMA interfaces on your board that are labelled DAC1, DAC2, ADC1, and ADC2. To setup your board for this demonstration, you can connect each channel in loopback as shown to the left of [Figure 1](#fig-1), or connect an antenna to one of the channels as shown to the right of [Figure 1](#fig-1).

Don't worry if you don't have an antenna. The default loopback configuration will still be very interesting and is connected as follows:
* Channel 0: DAC2 to ADC2
* Channel 1: DAC1 to ADC1

<a class="anchor" id="fig-1"></a>
<figure>
<img src='images/rfsoc2x2_setup.png' height='75%' width='75%'/>
    <figcaption><b>Figure 1: RFSoC2x2 development board setup, (left) loopback mode, (right) with an antenna.</b></figcaption>
</figure>

If you have chosen to use an antenna, you should connect channel 1 in loopback mode and connect the antenna to ADC2. The loopback connection will be useful for validating the initalisation of the spectrum analyzer. **Do Not** attach your antenna to any SMA interfaces labelled DAC1 or DAC2.

<div class="alert alert-box alert-danger">
<b>Caution:</b>
    In this demonstration, we generate tones using the RFSoC development board. Your device should be setup in loopback mode. You should understand that the RFSoC platform can also transmit RF signals wirelessly. Remember that unlicensed wireless transmission of RF signals may be illegal in your geographical location. Radio signals may also interfere with nearby devices, such as pacemakers and emergency radio equipment. Note that it is also illegal to intercept and decode particular RF signals. If you are unsure, please seek professional support.
</div>

### Software Setup <a class="anchor" id="software-setup"></a>
We're nearly finished setting up the demonstration system. The majority of the libraries used by the spectrum analyzer design are contained inside the RFSoC-SAM software package. We only need to run a few code cells to initialise the software environment.

The primary module for loading the Spectrum Analyzer design is contained inside `rfsoc_sam.overlay`. The class we are interested in using is `Overlay()`. During initialisation the class downloads the Spectrum Analyzer bitstream to the PL and configures the RF DCs and FPGA IP cores contained in our system. This process may take around a minute to complete.

**Run** the code cell below to load the RFSoC-SAM Overlay class.

In [ ]:
from rfsoc_sam.overlay import Overlay

sam = Overlay()

When the RFSoC-SAM Overlay class is initialising, the setup script will also program the LMK and LMX low-jitter clock chips on the RFSoC2x2 to 122.8MHz and 409.6MHz respectively.

Lets now initialise the analyzer, and setup user control.

In [ ]:
analyzer = sam.spectrum_analyzer()

----

## Simple Tone Generation <a class="anchor" id="simple-tone-generation"></a>
A simple amplitude controller is required to generate tones using the RF Digital-to-Analogue Converters (RF DACs). We use tone generation in this demonstration to provide a signal for the user to inspect when using the Spectrum Analyzer Module.

Run the code cell below to reveal a widget, which can be used to control the transmission frequency and amplitude.

In [ ]:
analyzer.children[2]

## The Spectrum Analyzer <a class="anchor" id="the-spectrum-analyzer"></a>
We will now explore the hardware accelerated Spectrum Analyzer Module. It is worthwhile noting the analyzers capabilities below:

* The analyzer is capable of inspecting 1638.4MHz of bandwidth.
* It can achieve a maximum spectral resolution of 0.244140625kHz. 
* The bandwidth is adjustable between 1638.4MHz and 1.6MHz.
* The range of inspection is between 0 to 4096MHz using higher order Nyquist techniques.

In [ ]:
analyzer.children[1]

The Spectrum Analyzer Module contains a hardware accelerated FFT core, which can convert the RF sampled signal to the frequency domain using a range of different FFT lengths, $N = 64$ upto $N = 8192$. The frequency domain signal is further manipulated using a custom floating point processor to obtain the representative Power Spectral Density (PSD) or Power Spectrum. Furthermore, a hardware accelerated decibel (dB) converter is also used to condition the frequency domain signal for visual analysis.

Through the loopback connection, you should be able to use the Spectrum Analyzer Module to locate the tone you previously generated using the tone generator. If you have an antenna connected to your board, try and locate signals of interest using the Spectrum Analyzer's control widgets.

### A Simple Example <a class="anchor" id="a-simple-example"></a>
If you would like to enable stimulus for the spectrum analyzer, you can use your mobile phone to create WiFi traffic. Follow the steps below to create an interesting WiFi spectrum to visualise.
* Connect your mobile phone to an access point that uses WiFi. 
* Then configure the spectrum analyzer for a centre frequency of 2400MHz and a decimation factor of 16. 
* Switch on the spectrum analyzer and spectrogram.
* Use your phone to stream a video, or music. This will create WiFi traffic for inspection.
* Place your phone close to the RF ADC ports of the spectrum analyzer.

You should see a similar output as given in the [Figure 2](#fig-2) below.

<a class="anchor" id="fig-2"></a>
<figure>
<img src='images/wifi_example.jpg' height='50%' width='50%'/>
    <figcaption><b>Figure 2: Capturing a WiFi signal using the Spectrum Analyser Module.</b></figcaption>
</figure>

## Conclusion <a class="anchor" id="conclusion"></a>

This notebook has presented a hardware accelerated Spectrum Analyzer Module for the RFSoC2x2 development board.